In [19]:
import pandas as pd
df= pd.read_csv("heart(in).csv")
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [20]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [21]:
df.duplicated().sum()

0

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [26]:
X = df.drop(["HeartDisease"], axis=1)  
y = df["HeartDisease"]

In [30]:
print(X.dtypes)


Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
dtype: object


In [32]:
X_encoded = pd.get_dummies(X)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [44]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=1, activation='sigmoid'))

C:\Users\sania\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [46]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True,
    verbose=1
)

In [54]:
checkpoint = ModelCheckpoint(
    'car_model.keras',
    monitor = 'val_loss',
    save_best_only=True,
    verbose=1
)

In [56]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 64)                  │           1,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [60]:
history = model.fit(X_train, y_train, batch_size=20, epochs=15, validation_split=0.3, callbacks=[early_stop, checkpoint], verbose=1)

Epoch 1/15
14/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5828 - loss: 0.6768
Epoch 1: val_loss improved from inf to 0.48157, saving model to car_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6304 - loss: 0.6434 - val_accuracy: 0.8190 - val_loss: 0.4816
Epoch 2/15
20/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8174 - loss: 0.4908 
Epoch 2: val_loss improved from 0.48157 to 0.40357, saving model to car_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8186 - loss: 0.4831 - val_accuracy: 0.8552 - val_loss: 0.4036
Epoch 3/15
19/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8231 - loss: 0.4233 
Epoch 3: val_loss improved from 0.40357 to 0.37070, saving model to car_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8283 - loss: 0.4170 - val_accuracy: 0.8507 - val_loss: 0.3707
Epoch 4/15
19/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8449 - loss: 0.4136
Epoch 4: val_loss improved from 0.37070 to 0.35876, saving model t

In [62]:
loss,accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8747 - loss: 0.3380
Test Accuracy: 0.8804


In [64]:
model.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


array([[0.04237407],
       [0.30568248],
       [0.97997576],
       [0.9398725 ],
       [0.04176585],
       [0.90585345],
       [0.90578914],
       [0.20012343],
       [0.69814724],
       [0.9062194 ],
       [0.46831125],
       [0.2147887 ],
       [0.63526833],
       [0.05403506],
       [0.8935006 ],
       [0.5826867 ],
       [0.055946  ],
       [0.9263113 ],
       [0.88506025],
       [0.10711747],
       [0.588371  ],
       [0.8666096 ],
       [0.03182903],
       [0.70063674],
       [0.93596476],
       [0.9425313 ],
       [0.03440268],
       [0.83742094],
       [0.06186463],
       [0.01534851],
       [0.79366916],
       [0.07476933],
       [0.62995726],
       [0.9735687 ],
       [0.92583406],
       [0.3229769 ],
       [0.96841854],
       [0.03836992],
       [0.94946265],
       [0.828143  ],
       [0.9519623 ],
       [0.9119566 ],
       [0.7432606 ],
       [0.13258977],
       [0.16290918],
       [0.6123996 ],
       [0.5642357 ],
       [0.934

In [66]:
checkpoint = ModelCheckpoint(
    'car_model.keras',
    monitor = 'val_precision',
    save_best_only=True,
    verbose=1
)

In [68]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 64)                  │           1,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,373 (40.52 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,916 (27.02 KB)

In [74]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['precision'])

In [76]:
history = model.fit(X_train, y_train, batch_size=20, epochs=15, validation_split=0.3, callbacks=[early_stop, checkpoint], verbose=1)

Epoch 1/15
17/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3459 - precision: 0.8647
Epoch 1: val_precision improved from inf to 0.82677, saving model to car_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.3426 - precision: 0.8673 - val_loss: 0.3575 - val_precision: 0.8268
Epoch 2/15
19/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2894 - precision: 0.8914 
Epoch 2: val_precision improved from 0.82677 to 0.82400, saving model to car_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2909 - precision: 0.8893 - val_loss: 0.3603 - val_precision: 0.8240
Epoch 3/15
17/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3428 - precision: 0.8628 
Epoch 3: val_precision did not improve from 0.82400
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3310 - precision: 0.8699 - val_loss: 0.3634 - val_precision: 0.8320
Epoch 4/15
24/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3228 - precision: 0.8728
Epoch 4: val_precision improved from 0.82400 to 0.81890, saving model to car_mo

In [80]:
loss,precision = model.evaluate(X_test, y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3360 - precision: 0.8961
